# Revised case normalization for Hirslanden Salem 2017

Notes: 
- There is no match cases to bfs database. Because the case_id `'41194601'` in the revised cases from DtoD, but the case_id in bfs database is '0041194601'
- What we did is to add 00 to the case_id in the revised case for this hospital



In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db
from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
file_info = FileInfo(
    os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
    'Hirslanden Salem',
    '2017',
    ['Änderungen_SA_2017']
)

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Salem', year='2017', sheets=['Änderungen_SA_2017'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

2022-10-14 09:45:56.557 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 26: discarded ICDs after validation {''}
2022-10-14 09:45:56.558 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 34: discarded ICDs after validation {''}
2022-10-14 09:45:56.559 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 43: discarded ICDs after validation {''}
2022-10-14 09:45:56.560 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 53: discarded ICDs after validation {''}
2022-10-14 09:45:56.563 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 0: discarded ICDs after validation {''}
2022-10-14 09:45:56.563 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 1: discarded ICDs after validation {''}
2022-10-14 09:45:56.564 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 2: discarded ICDs after validation {''}
2022-10-14 09:45:56.564 | DEBUG    | src.utils.dataframe_utils:_validate

Read 57 cases for Hirslanden Salem 2017
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


2022-10-14 09:45:56.574 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 21: discarded ICDs after validation {''}
2022-10-14 09:45:56.575 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 22: discarded ICDs after validation {''}
2022-10-14 09:45:56.577 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 23: discarded ICDs after validation {''}
2022-10-14 09:45:56.578 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 26: discarded ICDs after validation {''}
2022-10-14 09:45:56.579 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 28: discarded ICDs after validation {''}
2022-10-14 09:45:56.579 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 30: discarded ICDs after validation {''}
2022-10-14 09:45:56.580 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 32: discarded ICDs after validation {''}
2022-10-14 09:45:56.581 | DEBUG    | src.utils.dataframe_utils:_valid

In [ ]:
df_revised_case_d2d

# Compare the pd with new_pd, 
- If there are the same, we take the new_pd
- If there are not the same, we keep the new_pd. logging info: primary diagnosis has been changed
    - we check if the new_pd appears in the column of added_icds. If yes, remove from the added_icd list, because added_icd is for secondary diagnosis. 
    - we check if the pd appears in the column of removed_icds. If yes, remove from the removed_icd list, because removed_icd is for secondary diagnosis. 


In [ ]:
row_example = df_revised_case_d2d.loc[40]
row_example

In [ ]:
pd_col = 'old_pd'
pd_new_col = 'new_pd'
added_icd_col = 'added_icds'
removed_icd_col = 'removed_icds'

In [ ]:
def _validate_pd_revised_sd(row):
    pd_col = 'old_pd'
    pd_new_col = 'new_pd'
    added_icd_col = 'added_icds'
    removed_icd_col = 'removed_icds'
    old_pd = row[pd_col]
    new_pd = row[pd_new_col]
    added_icds = row[added_icd_col]
    removed_icds = row[removed_icd_col]
        # comparing old pd with new pd: discarding the new_pd if it is the same as the old pd.
    if old_pd != new_pd:
            
            # logger.debug(f'in the {row.name}, primary diagnosis where changed from {pd} to {new_pd}')
        if new_pd in added_icds:
            added_icds.remove(new_pd)
        if old_pd in removed_icds:
            removed_icds.remove(old_pd)
    row[pd_col] = new_pd
    row[added_icds] = added_icds
    row[removed_icd_col] = removed_icds

    return row

In [ ]:
_validate_pd_revised_sd(row_example)

In [ ]:
df_revised_case_d2d.apply(_validate_pd_revised_sd)

In [ ]:
def validate_pd_revised_sd(df: pd.DataFrame,
                        *,
                        pd_col: str = 'old_pd',
                        pd_new_col: str = 'new_pd',
                        added_icd_col: str = 'added_icds',
                        removed_icd_col: str = 'removed_icds',
                           ) -> pd.DataFrame:
    """Validate whether a list of old and new primary diagnosis are captured, discarding codes which are not correctly captured.
    Further: validation of added and removed secondary diagnosis (added_icds and removed icds),
    discarding those which do not correspond to an added or removed secondary diagnosis respectively.

    @param df: The data where to perform the filter.
    @param pd_col: The column containing the unrevised primary diagnosis to validate.
    @param pd_new_col: The column containing the revised primary diagnosis to validate.
    @param added_icd_col: The column containing the added secondary icds to validate.
    @param removed_icd_col: The column containing the removed secondary icds to validate.
    @return: The input DataFrame, with a validated 'pd' (consolidated column with 'pd' and 'pd_new'), 'added_icds',
    'removed_icds', overwriting existing columns
    """
    def _validate_pd_revised_sd(row):
        old_pd = row[pd_col]
        new_pd = row[pd_new_col]
        added_icds = row[added_icd_col]
        removed_icds = row[removed_icd_col]
        # comparing old pd with new pd: discarding the new_pd if it is the same as the old pd.
        if old_pd != new_pd:
            
            # logger.debug(f'in the {row.name}, primary diagnosis where changed from {pd} to {new_pd}')
            if new_pd in added_icds:
                added_icds.remove(new_pd)
            if old_pd in removed_icds:
                removed_icds.remove(old_pd)
        row[pd_col] = new_pd
        row[added_icds] = added_icds
        row[removed_icd_col] = removed_icds

        return row

    df = df.apply(_validate_pd_revised_sd, axis=1)
    return df

In [ ]:
validate_pd_revised_sd(df_revised_case_d2d, pd_col = 'old_pd',
                        pd_new_col = 'new_pd',
                        added_icd_col = 'added_icds',
                        removed_icd_col = 'removed_icds' )

In [ ]:
pd = row_example['pd']
new_pd = row_example['new_pd']
added_icds = row_example['added_icds']
removed_icds =  row_example['removed_icds']

if pd != new_pd:
    logger.debug(f"row {row.name}: discarded duplicated and invalid CHOP entries after validation {discard_chops}")
    logger.debug(f'in the {row.name}, primary diagnosis has been change from {pd} to {new_pd}')
    if new_pd in added_icds:
        added_icds.remove(new_pd)
    if pd in removed_icds:
        removed_icds.remove(pd)

row_example['pd'] = new_pd
row_example['added_icds'] = added_icds
row_example['removed_icds'] = removed_icds
print(row_example)    
    
    

In [ ]:
def validate_pd_revised_sd(df_revised_case_d2d):
    def _validate_pd_revised_sd(row):
        pd = row['pd']
        new_pd = row['new_pd']
        added_icds = row['added_icds']
        removed_icds =  row['removed_icds']
        if pd != new_pd:
            print(row.name)
            # logger.debug(f'in the {row.name}, primary diagnosis has been change from {pd} to {new_pd}')
            if new_pd in added_icds:
                added_icds.remove(new_pd)
            if pd in removed_icds:
                removed_icds.remove(pd)
        row['pd'] = new_pd
        row['added_icds'] = added_icds
        row['removed_icds'] = removed_icds

        return row
    df = df_revised_case_d2d.apply(_validate_pd_revised_sd)
    return df
    
validate_pd_revised_sd(df_revised_case_d2d)

In [22]:
_validate_pd_revised_sd(row_example)

/tmp/ipykernel_12214/2267993234.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[pd_col] = new_pd
/tmp/ipykernel_12214/2267993234.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[added_icds] = added_icds
/tmp/ipykernel_12214/2267993234.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[added_icds] = added_icds
/tmp/ipykernel_12214/2267993234.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

case_id                     41608546
patient_id          10553A6D7A39FBD7
gender                             M
age_years                         84
duration_of_stay                   3
pccl                               3
drg                             F62C
old_pd                         I5019
new_pd                         I5019
bfs_code                        M100
added_icds                        []
removed_icds                      []
added_chops                       []
removed_chops                     []
Name: 40, dtype: object

In [23]:
df_revised_case_d2d.apply(_validate_pd_revised_sd)

KeyError: 'old_pd'

In [8]:
def validate_pd_revised_sd(df: pd.DataFrame,
                        *,
                        pd_col: str = 'old_pd',
                        pd_new_col: str = 'new_pd',
                        added_icd_col: str = 'added_icds',
                        removed_icd_col: str = 'removed_icds',
                           ) -> pd.DataFrame:
    """Validate whether a list of old and new primary diagnosis are captured, discarding codes which are not correctly captured.
    Further: validation of added and removed secondary diagnosis (added_icds and removed icds),
    discarding those which do not correspond to an added or removed secondary diagnosis respectively.

    @param df: The data where to perform the filter.
    @param pd_col: The column containing the unrevised primary diagnosis to validate.
    @param pd_new_col: The column containing the revised primary diagnosis to validate.
    @param added_icd_col: The column containing the added secondary icds to validate.
    @param removed_icd_col: The column containing the removed secondary icds to validate.
    @return: The input DataFrame, with a validated 'pd' (consolidated column with 'pd' and 'pd_new'), 'added_icds',
    'removed_icds', overwriting existing columns
    """
    def _validate_pd_revised_sd(row):
        old_pd = row[pd_col]
        new_pd = row[pd_new_col]
        added_icds = row[added_icd_col]
        removed_icds = row[removed_icd_col]
        # comparing old pd with new pd: discarding the new_pd if it is the same as the old pd.
        if old_pd != new_pd:
            
            # logger.debug(f'in the {row.name}, primary diagnosis where changed from {pd} to {new_pd}')
            if new_pd in added_icds:
                added_icds.remove(new_pd)
            if old_pd in removed_icds:
                removed_icds.remove(old_pd)
        row[pd_col] = new_pd
        row[added_icds] = added_icds
        row[removed_icd_col] = removed_icds

        return row

    df = df.apply(_validate_pd_revised_sd, axis=1)
    return df

In [10]:
validate_pd_revised_sd(df_revised_case_d2d, pd_col = 'old_pd',
                        pd_new_col = 'new_pd',
                        added_icd_col = 'added_icds',
                        removed_icd_col = 'removed_icds' )

KeyError: "['N183'] not in index"

In [ ]:
pd = row_example['pd']
new_pd = row_example['new_pd']
added_icds = row_example['added_icds']
removed_icds =  row_example['removed_icds']

if pd != new_pd:
    logger.debug(f"row {row.name}: discarded duplicated and invalid CHOP entries after validation {discard_chops}")
    logger.debug(f'in the {row.name}, primary diagnosis has been change from {pd} to {new_pd}')
    if new_pd in added_icds:
        added_icds.remove(new_pd)
    if pd in removed_icds:
        removed_icds.remove(pd)

row_example['pd'] = new_pd
row_example['added_icds'] = added_icds
row_example['removed_icds'] = removed_icds
print(row_example)    
    
    

In [ ]:
def validate_pd_revised_sd(df_revised_case_d2d):
    def _validate_pd_revised_sd(row):
        pd = row['pd']
        new_pd = row['new_pd']
        added_icds = row['added_icds']
        removed_icds =  row['removed_icds']
        if pd != new_pd:
            print(row.name)
            # logger.debug(f'in the {row.name}, primary diagnosis has been change from {pd} to {new_pd}')
            if new_pd in added_icds:
                added_icds.remove(new_pd)
            if pd in removed_icds:
                removed_icds.remove(pd)
        row['pd'] = new_pd
        row['added_icds'] = added_icds
        row['removed_icds'] = removed_icds

        return row
    df = df_revised_case_d2d.apply(_validate_pd_revised_sd)
    return df
    
validate_pd_revised_sd(df_revised_case_d2d)

In [ ]:
df_revised_case_d2d['pd'] == df_revised_case_d2d['new_pd']

# match to the database


In [ ]:
revised_case_id

In [ ]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

In [ ]:
# change the case_id the same format as bfs data e.g. '0041194601'
revised_case_id_pad0 = [case_id.zfill(10) for case_id in revised_case_id]
revised_case_id_pad0[0]

In [ ]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id_pad0)
revised_case_db.head()

In [ ]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file_info.hospital_name_db, file_info.year))

In [ ]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match

In [ ]:
# substract the columns [case_id, new_pd_bfs_code	added_icds	removed_icds	added_chops	removed_chops] from revised_cases 


revised_case_